# Proper Orthogonal Decomposition for Reduced Order Modelling 

<a id="table"></a>

## Table of contents
- [Introduction](#intro)
- [High resolution model](#HR)
- [Proper orthogonal decomposition](#POD)
- [Low resolution model](#LR)
- [Performance improvement](#improvement)
- [Error between high and low resolution models](#error)
- [References](#ref)

In [2]:
from time import process_time
import numpy as np
from scipy.linalg import svd
from scipy.sparse import csr_matrix

from modules.rom import pod
from scripts import animation
# reservoir model
from scripts import water_canals_model as model
from modules.simulator import simplots
from modules.simulator.units import UnitRegistry

# Units
u = UnitRegistry()

# Set image folder path
imgpath = ".\\images"

<a id="intro"></a>

## [Introduction](#table)

 Reduced order modeling (ROM) has the aim of reducing the number of dimensions (equations) that represent a model in order to solve it faster. In this notebook I show an example of how to do ROM with proper orthogonal decomposition (POD). I follow the procedure presented in this paper [SPE-141922-MS](#ref).

<a id="HR"></a>

## [High resolution model (HRM)](#table)

The model is a single layer of rock of low permeability with two canals of high permeability. The fluid is water. And there is a difference of pressure of 200 psi between the left face and right face of the rock layer. [This image](#perm) shows the permeability distribution in the rock.

The model has a grid with 5,000 cells and it is solved with the Lagging Coefficients method. 

The full simulation schedule is 500 timesteps that add up to 10 days. The training simulation consists of the first 100 timesteps of that schedule.

In [6]:
# Plot permeability
log_perm = np.log(model.rock.perm/u.darcy / u.milli)
simplots.plotCellValues2D(model.grid, log_perm, 'inferno', np.min(log_perm), np.max(log_perm),
                     title='Permeability Ln(mD)', 
                     filename='{}\\pod_perm'.format(imgpath))

<a id="perm">
        <img src=".\images\pod_perm.png?" style="width:400px;height:-1px;" >
</a> 

In [7]:
t = process_time()
# Run simulation
X, well_solution, sch = model.LCM.solve(model.sch, max_inner_iter = 1, tol = 1E-6, ATS = False)
HRM_time = process_time() - t
model.nice_print('Time running simulation = {:.3f} seconds.'.format(HRM_time))

 Time running simulation = 133.740 seconds.

The following animation shows the pressure distribution in the rock layer. As it is expected, the pressure increases first in the high permeability area of the canals.
<img src=".\images\pod_pw.gif" >

<a id = POD></a>

## [Proper orthogonal decomposition (POD)](#table)

In [34]:
# Snapshots
D = X[:,:100]
# Covariance matrix
C = D.T.dot(D)
# Singular value decomposition
V,si,_ = svd(C)
# Calculate basis
U = (D.dot(V)) * si ** (1/2)
print(U.shape)

(5000, 100)


The matrix $U$ contains the basis vectors. To form the reduced basis matrix $\:\phi$, only the basis vectors that contribute the most energy are selected. The following plots show that the first vector captures almost all of the energy. Also, there is a change of slope at vector 15. 

In [35]:
pod.plot_energy(si, "{}\\pod_energy".format(imgpath))
pod.plot_energy(si[:40], "{}\\pod_energy_closeup".format(imgpath))

**Energy of the basis vectors**
<img src = ".\images\pod_energy.png?" style="width:400px;height:-1px;" >
**Close up**
<img src = ".\images\pod_energy_closeup.png?" style="width:400px;height:-1px;" >

In [36]:
basis_size = 5
reduced_basis = U[:,:basis_size]

<a id="LR"></a>

## [Low resolution model (LRM)](#table)

In [37]:
linear_solver = lambda A, b : pod.linear_solver(A, b, csr_matrix(reduced_basis))

In [38]:
t = process_time()
# Run simulation
Xr, well_solution, sch = model.LCM.solve(model.sch, max_inner_iter = 1, tol = 1E-6, ATS = False, linear_solver = linear_solver)
LRM_time = process_time() - t
model.nice_print('Time running simulation = {:.3f} seconds.'.format(LRM_time))

 Time running simulation = 105.067 seconds.

The solution of the simulation with the POD technique.

<img src=".\images\pod_LRM_pw.gif" >

<a id="improvement"></a>

## [Performance improvement](#table)

There is a performance improvement in the total simulation time of almost 22%.

In [39]:
improvement = (HRM_time - LRM_time )/ HRM_time 
print('Save {:.2f}% of the time.'.format(improvement * 100))

Save 21.44% of the time.


<a id="error"></a>

## [Error between high and low resolution models](#table)

The next animation shows the percentage difference between the high resolution and low resolution solutions for each cell for each timestep. The maximum difference between solutions is approximately 4.5 %. 

In [40]:
# X is the solution from the HRM and Xr is the solution from the LRM
diff = np.subtract(X/u.psi,Xr/u.psi)
diff = np.divide(diff,X/u.psi) * 100
max_diff = (np.max(np.abs(diff)))
print('Maximum relative error: {:.2f}%.'.format(max_diff))
print('Average error : {:.2f}% '.format(np.mean(np.abs(diff))))

Maximum relative error: 4.48%.
Average error : 0.46% 


<img src = ".\images\pod_error.gif" style="width:400px;height:-1px;" >

### Animations
Change the following cells to "Code" to run the code and make the animations. It takes some time to run.

<a id="ref"></a>

## [References](#table)

Astrid, P., Papaioannou, G., Vink, J. C., & Jansen, M. D. (2011, January 1). Pressure Preconditioning Using Proper Orthogonal Decomposition. Society of Petroleum Engineers. doi:10.2118/141922-MS

## [To the top](#table)

In [46]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

# Using the style sheet found here  Lorena Barba /* https://github.com/barbagroup/CFDPython/blob/master/styles/custom.css */